In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from scipy.stats import linregress

import matplotlib.pyplot as plt
%matplotlib widget

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.style.use("paper")
figpath = "/Users/johnrussell/Documents/figures/2023-06-figs-paper/"
savefigs=False

In [ ]:
def exp_approach(x, a, k):
    return a*(1-np.exp(-k*x))#+b

def exp_approach2(x, a, k,t):
    return a*(1-np.exp(-k*(x-t)))#+b

def exp_approach3(x, a, k,b):
    return a*(1-np.exp(-k*x))+b

def exp_decay(x, a,k):
    return a*np.exp(-k*x)#+b

In [ ]:
def hyp(x, a, b):
    return a*x/(b+x)

In [ ]:
def hyp_decay(x, a, b):
    return a/(b+x)

In [ ]:
# x = np.linspace(0, 10, 51)
# ye = 5*(1-np.exp(-0.35*x))
# yh = 6.5*x/(3+x)
# plt.figure()
# plt.plot(x, ye)
# plt.plot(x, yh)

In [ ]:
wt_file = "/Users/johnrussell/Data/2023-05-16/dh229_srs_tables.h5"
dh245_file = "/Users/johnrussell/Data/2023-05-01-combined/dh245_srs_tables.h5"
dh246_file = "/Users/johnrussell/Data/2023-05-01-combined/dh246_dox_srs_tables"
dil_file = "/Users/johnrussell/Data/2023-05-16/dh229_dilution_srs_tables.h5"

In [ ]:
wt_dt = (pd.read_hdf("/Users/johnrussell/Data/2023-05-16/2023-05-16-timestamps.h5")
         .astype({'RelativeTime':'f4'})
         .groupby(['S','T']).datetime.min()
         .groupby('S').diff()
         .mean().total_seconds()/60)

In [ ]:
wt_dim_data = pd.read_hdf("/Users/johnrussell/Data/2023-05-16/2023-05-16-dimdata.h5")
vox_area = float(wt_dim_data.loc[0,('Voxel', 'X')])**2

In [ ]:
wt_area = vox_area*pd.read_hdf(wt_file, key='area')
dh245_area = vox_area*pd.read_hdf(dh245_file, key='area')
dh246_area = vox_area*pd.read_hdf(dh246_file, key='area')

In [ ]:
# wt_area[t_plt].mean()/vox_area

# dh245_area[t_plt].mean()/vox_area- wt_area[t_plt].mean()/vox_area

# wt_area[t_plt].mean() - dh245_area[t_plt].mean()

In [ ]:
plt.figure()
t_plt= 20 
a = wt_area[t_plt].dropna().values
counts, bins, patches = plt.hist(a, bins=46, range=(0,45), density=True, alpha=0.65, label=f"WT (N={a.shape[0]})")
a = dh245_area[t_plt].dropna().values
_ = plt.hist(dh245_area.values[:,t_plt], bins=bins, density=True, alpha=0.65, label=f"HXT2 Only (N={a.shape[0]})")
# _ = plt.hist(dh246_area.values[:,t_plt], bins=bins, density=True, alpha=0.65, label="HXT5 Only")
plt.legend()
plt.title("Cell Areas")
plt.xlabel("Area $\\left(\mu m^2\\right)$")
plt.ylabel("Fraction of cells")
if savefigs: plt.savefig(figpath+"area_comparison.png")

In [ ]:
wt_traces = pd.read_hdf(wt_file, key='recon')
dh245_traces = pd.read_hdf(dh245_file, key='recon')
dh246_traces = pd.read_hdf(dh246_file, key='recon')
dil_traces = pd.read_hdf(dil_file, key='recon')

In [ ]:
bulk_wt = wt_traces.mean()
bulk_dh245 = dh245_traces.mean()
bulk_dh246 = dh246_traces.mean()
bulk_dil = dil_traces.mean()

In [ ]:
t_wt = np.arange(wt_traces.shape[1])*wt_dt#+28.75

In [ ]:
sigma = wt_traces.sem().values
# y_dil = keep_dil.mean().values

y = bulk_wt.values
y_offset = y[0]
y = y-y_offset
fit_e = curve_fit(exp_approach, t_wt, y, sigma=sigma, p0=[20, 0.05], bounds=(0,np.inf))
fit_h= curve_fit(hyp, t_wt, y, sigma=sigma, p0=[20, 10], bounds=(0,np.inf))

a,k = fit_e[0]
xplt = np.linspace(0, 520, 250)
plt.figure()
plt.plot(xplt, exp_approach(xplt, *fit_e[0])+y_offset, zorder=0, linewidth=3, alpha=0.75)
plt.plot(xplt, hyp(xplt, *fit_h[0])+y_offset, zorder=0, linewidth=3, alpha=0.75)
# plt.plot(x, 26*x/(10+x))
plt.errorbar(t_wt, bulk_wt.values, yerr=sigma, fmt='k.',capsize=2, markersize=1.5, zorder=-1)
plt.title(f"Wild Type: y={a:0.3f}(1-exp(-{k:0.3f}x))")
if savefigs:
    plt.savefig(fig_path+"bulk_fit.png")

In [ ]:
rmse_e = np.sqrt(((y- exp_approach(t_wt, *fit_e[0]))**2).mean())
rmse_h = np.sqrt(((y - hyp(t_wt, *fit_h[0]))**2).mean())

In [ ]:
print(f"{rmse_e=:0.4g}")
print(f"{rmse_h=:0.4g}")

In [ ]:
plt.figure()
for i,df in enumerate([wt_traces, dh245_traces, dh246_traces]):
    y = df.mean().values
    y_offset = y[0]
    y = y-y_offset
    sy = df.sem().values
    fit = curve_fit(exp_approach, t_wt, y, sigma=sy, p0=[20, 0.05], bounds=(0,np.inf))
    # fit1 = curve_fit(hyp, t_wt, bulk_wt.values, sigma=sigma, p0=[20, 10, 10], bounds=(0,np.inf))
    plt.errorbar(t_wt, y+y_offset, yerr=sy, fmt='.', capsize=3)
    plt.plot(xplt, exp_approach(xplt, *fit[0])+y_offset, color=colors[i])
plt.title("exponential fits")

In [ ]:
plt.figure()
for i,df in enumerate([wt_traces, dh245_traces]):
    y = df.mean().values
    y_offset = y[0]
    y = y-y_offset
    sy = df.sem().values
    fit_e = curve_fit(exp_approach, t_wt, y, sigma=sy, p0=[20, 0.05], bounds=(0,np.inf))
    fit_h = curve_fit(hyp, t_wt, y, sigma=sy, p0=[1000, 100], bounds=(0,np.inf))
    rmse_e = np.sqrt(((y - exp_approach(t_wt, *fit_e[0]))**2).mean())
    rmse_h = np.sqrt(((y - hyp(t_wt, *fit_h[0]))**2).mean())
    print(i, f"{rmse_e=:0.4f}",  f"{rmse_h=:0.4f}")
    plt.plot(t_wt, y, '.')
    plt.plot(xplt, hyp(xplt, *fit_h[0]), color=colors[i])
    plt.plot(xplt, exp_approach(xplt, *fit_e[0]), ":", color=colors[i])

In [ ]:
df = dh245_traces
y = df.mean().values
sy = df.sem().values
plt.figure()
plt.errorbar(t_wt, y, yerr=sy, fmt='k.', capsize=3)
y_offset = y[0]
fit_e = curve_fit(exp_approach, t_wt[1:], y[1:]-y_offset, p0=[20, 0.05], bounds=(0,np.inf))
fit_h = curve_fit(hyp, t_wt[1:], y[1:]-y_offset, p0=[1000, 100], bounds=(0,np.inf))

rmse_e = np.sqrt(((y-y_offset - exp_approach(t_wt, *fit_e[0]))**2).mean())
rmse_h = np.sqrt(((y-y_offset - hyp(t_wt, *fit_h[0]))**2).mean())
plt.plot(xplt, exp_approach(xplt, *fit_e[0])+y_offset, '--',
         color=colors[0], alpha=0.7, label=f'Exponential fit: y={fit_e[0][0]:0.3g} (1-exp[-{fit_e[0][1]:0.3g}t]\n RMSE={rmse_e:0.3g}')
plt.plot(xplt, hyp(xplt, *fit_h[0])+y_offset, color=colors[0], alpha=0.7,
         label=f'Hyperbolic fit: y={fit_h[0][0]:0.3g}t/ ({fit_h[0][1]:0.3g}+t)\n RMSE={rmse_h:0.3g}')
plt.title("Different fits to HXT2-only Net Glucose Uptake")
plt.xlabel("Time (minutes)")
plt.ylabel("SRS Intensity (a.u.)")
plt.legend()
if savefigs: plt.savefig(figpath+"fit_comparison.png")

In [ ]:
14.5*0.00514

In [ ]:
19.9/230

In [ ]:
(rmse_e-rmse_h)/rmse_e

In [ ]:
plt.figure()
for i,df in enumerate([wt_traces, dil_traces]):
    y = df.mean().values[:36]
    plt.plot(t_wt, y, '.')
    y0 = y.max()+3
    #y_offset = y[int(-1*i)]
    # y = y-y_offset
    #sy = df.sem().values[:36]
    y = -1*y0*(i-1)+(2*i-1)*y
    fit_l = linregress(t_wt, np.log(y))
    if i==0:
        # y = 25 - y
        fh = hyp
        fe = exp_approach
    if i==1:
        fit_l = linregress(t_wt, np.log(y))
        fh = hyp_decay
        fe = exp_decay
        
    fit_e = curve_fit(fe, t_wt, y, sigma=sy, p0=[20, 0.05], bounds=(0,np.inf))
    # fit_h = curve_fit(fh, t_wt, y, sigma=sy, p0=[1000, 100], bounds=(0,np.inf))
    # plt.plot(xplt, fh(xplt, *fit_h[0])+y_offset, color=colors[i])
    # plt.plot(xplt, fe(xplt, *fit_e[0])+y_offset, ":", color=colors[i])
    plt.plot(xplt, -1*y0*(i-1)+(2*i-1)*np.exp(fit_l.intercept+fit_l.slope*xplt), color=colors[i])
    # rmse_e = np.sqrt(((y - fe(t_wt, *fit_e[0]))**2).mean())
    # rmse_h = np.sqrt(((y - fh(t_wt, *fit_h[0]))**2).mean())
    # print(i, f"{rmse_e=:0.4f}",  f"{rmse_h=:0.4f}")
    print(i, f"{fit_l.slope:0.3g}",f"{np.exp(fit_l.intercept):0.3g}")

# plt.semilogy()

In [ ]:
plt.figure()
fits = []
for i,df in enumerate([wt_traces, dh245_traces]):
    scaled = ((df-df[0].mean()))
    y = scaled.mean().values
    sy = scaled.sem().values
    fit = curve_fit(exp_approach, t_wt, y, sigma=sigma, p0=[20, 0.05], bounds=(0,np.inf))
    fits.append(fit)
    name = ["WT", "HXT2-only"]
    plt.plot(xplt, exp_approach(xplt, *fit[0]), color=colors[i], label=name[i]+f": {fit[0][0]:0.3g} (1-exp[-{fit[0][1]:0.3g}t])")
    # fit1 = curve_fit(hyp, t_wt, bulk_wt.values, sigma=sigma, p0=[20, 10, 10], bounds=(0,np.inf))
    plt.errorbar(t_wt, y, yerr=sy, fmt='.', capsize=3, color=colors[i])
plt.legend()
plt.title("HXT2-only glucose uptake vs WT")
plt.xlabel("Time (minutes)")
plt.ylabel("SRS (a.u.)")
plt.savefig(figpath +"hxt2_vs_wt.png")

In [ ]:
np.prod(fits[0][0])

In [ ]:
np.prod(fits[1][0])

In [ ]:
plt.figure()
plt.plot(t_wt, (~(dh245_traces.isna())).sum())

In [ ]:
22.9*.0038

In [ ]:
wt_params = pd.read_hdf(wt_file, key='params')
dh245_params = pd.read_hdf(dh245_file, key='params')
dil_params = pd.read_hdf(dil_file, key='params')

In [ ]:
wt_params['p'] = wt_params['a']*wt_params['k']
dh245_params['p'] = dh245_params['a']*dh245_params['k']
dil_params['p'] = dil_params['a']*dil_params['k']

In [ ]:
ranges = {'k':(0,0.075), 'a':(0,75)}#, 'p':(0,1.5)}

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(6.52, 3.5))
for i,(x,hrng) in enumerate(ranges.items()):
    counts, bins, patches = ax[i].hist(wt_params[x], bins=50, range=hrng, density=True, alpha=0.6)
    # ax[i].hist(dh245_params[x], bins=bins, density=True, alpha=0.6);
    ax[i].hist(dil_params[x], bins=bins, density=True, alpha=0.6);
    ax[i].set_xlabel(x);

In [ ]:
wt_count = pd.read_hdf(wt_file, key='count')
dil_count = pd.read_hdf(dil_file, key='count')

In [ ]:
plt.figure()
lrs=[]
for i,a in enumerate([wt_count]):#, dil_count]):
    # count = a.stack('T').loc[a.stack('T')>0].groupby('T').count().values[:36]
    y = a.values[:36]
    lr = linregress(t_wt, np.log(y))
    lrs.append(lr)
    s = ['Deuterated', 'Unlabeled'][i]
    plt.plot(t_wt, y, '.', label=f"Growth rate = {lr.slope:0.4g}"+" min$^{-1}$", color=colors[i])
    # plt.plot(xplt, 2**(lr.intercept+lr.slope*xplt), color=colors[i], alpha=0.6)
    plt.plot(xplt, np.exp((lr.intercept+lr.slope*xplt)), color=colors[i], alpha=0.6)
plt.semilogy()
plt.title("Effect of Deuterated Medium on Growth")
plt.xlabel("Time (minutes)")
plt.ylabel("Number of cells")
plt.legend();
if savefigs: plt.savefig(figpath+"wt_growth_rate.png")

In [ ]:
from tqdm.autonotebook import tqdm

In [ ]:
def fit_exp(x, y, p0):
    try:
        p, cov = curve_fit(exp_approach, x, y, p0=p0, bounds=(0,np.inf), absolute_sigma=True)
        out = np.array([*p, *np.diag(cov)])
    except RuntimeError:
        out = np.full(2*len(p0), np.nan)
    return out

In [ ]:
# slower to use dask delayed by ~50%
df = pd.DataFrame(wt_traces)
t = t_wt

wt_params = pd.DataFrame(index=df.index, columns=['a','k', 'sig_a', 'sig_k','x0','y0'], dtype='f4')
for i, (idx, s) in enumerate(tqdm(df.iterrows(), total=len(df))):
    y = s.values
    mask = ~np.isnan(y)
    y = y[mask]
    y_offset = y[0]
    y = y-y_offset
    x = t_wt[mask]
    x_offset = x[0]
    x = x - x_offset
    out = fit_exp(x,y,fit[0])
    wt_params.loc[idx] = [*out, x_offset, y_offset]
wt_params = wt_params.dropna()   

In [ ]:
# slower to use dask delayed by ~50%
df = pd.DataFrame(dh245_traces)
t = t_wt

dh245_params = pd.DataFrame(index=df.index, columns=['a','k', 'sig_a', 'sig_k','x0','y0'], dtype='f4')
for i, (idx, s) in enumerate(tqdm(df.iterrows(), total=len(df))):
    y = s.values
    mask = ~np.isnan(y)
    y = y[mask]
    y_offset = y[0]
    y = y-y_offset
    x = t_wt[mask]
    x_offset = x[0]
    x = x - x_offset
    out = fit_exp(x,y,fit[0])
    dh245_params.loc[idx] = [*out, x_offset, y_offset]
dh245_params = dh245_params.dropna()   

In [ ]:
df = pd.DataFrame(wt_params)
pcts = np.linspace(0,1, 101)
qs = df.quantile(pcts)

sig_k_sel = df['sig_k'].between(*qs.loc[[0.02, 0.98],'sig_k'])#.between(1e-16, 1e4)
sig_a_sel = df['sig_a'].between(*qs.loc[[0.02, 0.98],'sig_a'])#.between(1e-16,1e5)#.between(1e-16,10)

fig, ax = plt.subplots(1,2, figsize=(6.52, 2.5))
df = df.loc[sig_k_sel&sig_a_sel]
hranges = ([0,0.05], [0,50])
for i,x in enumerate('ka'):
    # w = 1./df[f'sig_{x}']
    # w /= w.sum()
    
    counts, bins, patches = ax[i].hist(df[x].values, bins=100, range=hranges[i])
    print(x, bins[np.argmax(counts)], df[x].mean())
    ax[i].axvline(df[x].median(), color='k', alpha=0.5, label=f'Median = {df[x].median():0.3g}')
    ax[i].legend()
ax[0].set_xlabel(r"Uptake Rate  $\left(min^{-1}\right)$")
ax[1].set_xlabel(r"Amplitude $\left( a.u. \right)$")

ax[0].set_ylabel("Cell counts")

fig.suptitle("Estimated Single-Cell Metabolic Parameters")
# if savefigs:
#     plt.savefig(fig_path+"sc_params.png")

In [ ]:
df = pd.DataFrame(dh245_params)
pcts = np.linspace(0,1, 101)
qs = df.quantile(pcts)

sig_k_sel = df['sig_k'].between(*qs.loc[[0.02, 0.98],'sig_k'])#.between(1e-16, 1e4)
sig_a_sel = df['sig_a'].between(*qs.loc[[0.02, 0.98],'sig_a'])#.between(1e-16,1e5)#.between(1e-16,10)

fig, ax = plt.subplots(1,2, figsize=(6.52, 2.5))
df = df.loc[sig_k_sel&sig_a_sel]
hranges = ([0,0.05], [0,50])
for i,x in enumerate('ka'):
    
    counts, bins, patches = ax[i].hist(df[x].values, bins=100, range=hranges[i])
    print(x, bins[np.argmax(counts)], df[x].mean())
    ax[i].axvline(df[x].median(), color='k', alpha=0.5, label=f'Median = {df[x].median():0.3g}')
    ax[i].legend()
ax[0].set_xlabel(r"Uptake Rate  $\left(min^{-1}\right)$")
ax[1].set_xlabel(r"Amplitude $\left( a.u. \right)$")

ax[0].set_ylabel("Cell counts")

fig.suptitle("HXT2-Only Single-Cell Metabolic Parameters")
if savefigs:
    plt.savefig(figpath+"dh245_sc_params.png")

In [ ]:
plt.figure()
for i,df in enumerate([wt_params, dh245_params]):
    pcts = np.linspace(0,1, 101)
    qs = df.quantile(pcts)

    sig_k_sel = df['sig_k'].between(*qs.loc[[0.02, 0.98],'sig_k'])#.between(1e-16, 1e4)
    sig_a_sel = df['sig_a'].between(*qs.loc[[0.02, 0.98],'sig_a'])#.between(1e-16,1e5)#.between(1e-16,10)

    df = df.loc[sig_k_sel&sig_a_sel]
    l = ['WT', 'HXT2-Only']
    x = df['a']*df['k']
    plt.hist((x).values, range=(0,0.5),bins=41, density=True, alpha=0.6, label=f"{l[i]}: median = {x.median():0.3f}")
    # plt.hist((x).values, range=(0,0.5),bins=41, density=True, alpha=0.6, label=f"{l[i]} - median = {x.median():0.3f}")
plt.legend()

In [ ]:
first = wt_traces.stack('T').reset_index('T').groupby(['CellID', 'S']).first('T').set_index('T', append=True)

In [ ]:
params_by_age = df.join(first.reset_index('T')['T']).groupby('T').mean()
params_by_age_sem = df.join(first.reset_index('T')['T']).groupby('T').sem()

In [ ]:
df = pd.DataFrame(wt_params)
pcts = np.linspace(0,1, 101)
qs = df.quantile(pcts)

sig_k_sel = df['sig_k'].between(*qs.loc[[0.02, 0.98],'sig_k'])#.between(1e-16, 1e4)
sig_a_sel = df['sig_a'].between(*qs.loc[[0.02, 0.98],'sig_a'])#.between(1e-16,1e5)#.between(1e-16,10)

df = df.loc[sig_k_sel&sig_a_sel]
df = df.loc[df.k>1e-6]

first = wt_traces.stack('T').reset_index('T').dropna().groupby(['S','CellID']).first().set_index('T',append=True)
params_by_age = df.join(first.reset_index('T')['T']).groupby('T').mean()
params_by_age_sem = df.join(first.reset_index('T')['T']).groupby('T').sem()

fig, ax = plt.subplots(1,2, figsize=(6.52,3))
ax[0].plot(df['k'], np.sqrt(df['sig_k']), 'k.', markersize=1)
xlog = np.logspace(np.log10(df['k'].min()), np.log10(df['k'].max()))
for m in np.logspace(-3, 0, 4, base=2 ):# [0.05,0.1, 0.5, 1]: #np.logspace(-1.5,1,5):

    y = m*xlog
    x = xlog[y>np.sqrt(df['sig_k'].min())]
    y = y[y>np.sqrt(df['sig_k'].min())]
    ax[0].plot(x,y, alpha=0.6, label="$\sigma_k =$"+f"{m:0.2f} k")
ax[0].set_xlabel("k")
ax[0].set_ylabel("$\sigma_k$")
ax[0].loglog()
ax[0].set_title("Larger Errors with Larger $k$")
ax[0].legend()
           
ax[1].errorbar(t_wt[:22], params_by_age['k'], params_by_age_sem['k'], fmt='o', capsize=3)
ax[1].set_ylabel("k")
ax[1].set_xlabel("Time of birth")
ax[1].set_title("Larger k values with younger cells")
# plt.savefig(figpath+"rate_errors.png")

In [ ]:
sk_sel = wt_params.loc[(np.sqrt(wt_params['sig_k'])/wt_params['k'])<0.25]

In [ ]:
sk_sel.median()

In [ ]:
first = wt_traces.stack('T').reset_index('T').dropna().groupby(['S','CellID']).first().set_index('T',append=True)

params_by_age = df.join(first.reset_index('T')['T']).groupby('T').mean()
params_by_age_sem = df.join(first.reset_index('T')['T']).groupby('T').sem()

In [ ]:
pcts = np.linspace(0,1, 101)
qs = wt_params.quantile(pcts)

In [ ]:
plt.figure()
tc = 3
plt.plot(pcts, qs.sig_a.values, '-o', markersize=2)
plt.plot(pcts, qs.sig_k.values,'-o', markersize=2)
plt.semilogy();

In [ ]:
plt.figure()
a = wt_count
y = a.values
lr = linregress(t_wt, np.log(y))
lr2 = linregress(t_wt, np.log2(y))
plt.plot(t_wt, y, '.', label=f"Growth rate $\gamma$ = {lr.slope:0.4g} "+"min$^{-1}$", color=colors[0])
plt.plot(xplt, np.exp((lr.intercept+lr.slope*xplt)), color=colors[0], alpha=0.6)
# plt.plot(xplt, np.exp(np.log(2)*(lr2.intercept+lr2.slope*xplt)), color=colors[1], alpha=0.6, label=f"Growth rate: µ = {lr2.slope:0.4g}")
plt.semilogy()
plt.title("Wild Type Population Growth")
plt.xlabel("Time (minutes)")
plt.ylabel("Number of cells (log scale)")
plt.legend();
if savefigs: plt.savefig(figpath+"wt_growth_rate.png")

In [ ]:
def fit_exp_decay(x, y, p0):
    try:
        p, cov = curve_fit(exp_decay, x, y, p0=p0, bounds=(0,np.inf), absolute_sigma=True)
        out = np.array([*p, *np.diag(cov)])
    except RuntimeError:
        out = np.full(2*len(p0), np.nan)
    return out

In [ ]:
## Single Cell Analysis

df = dil_traces.iloc[:,:36]
t = t_wt[:36]

dil_params = pd.DataFrame(index=df.index, columns=['a','k','sig_a', 'sig_k','x0', 'y0'], dtype='f4')

for i, (idx, s) in enumerate(tqdm(df.iterrows(), total=len(df))):
    y = s.values
    mask = ~np.isnan(y)
    y = y[mask]
    y_offset = y[-1]
    y = y#-y_offset
    x = t[mask]
    x_offset = x[0]
    x = x-x_offset
    out = fit_exp_decay(x,y,fit[0])
    p = [*out, x_offset, y_offset]
    dil_params.loc[idx] = p

# unconverged = all_srs_params.loc[all_srs_params.isna().any(axis=1)].index
# dil_params = dil_params.dropna()

In [ ]:
# Single Cell Analysis

df = dil_traces.iloc[:,:36]
t = t_wt[:36]
dil_params_lin = pd.DataFrame(index=df.index, columns=['a','k','sig_a', 'sig_k'], dtype='f4')

for i, (idx, s) in enumerate(tqdm(df.iterrows(), total=len(df))):
    y = s.values
    # mask = ~np.isnan(y)
    mask = y>0
    y = y[mask]
    x = t[mask]
    lr = linregress(x, np.log(y))
    p = [np.exp(lr.intercept), lr.slope, lr.intercept_stderr, lr.stderr]
    dil_params_lin.loc[idx] = p

# unconverged = all_srs_params_lin.loc[all_srs_params.isna().any(axis=1)].index
dil_params_lin = dil_params_lin.dropna()

In [ ]:
fig, ax = plt.subplots(3,2, figsize=(6.52, 5))

for n, df in enumerate([dil_params, dil_params_lin, wt_params]):
    sig_k_sel = df['sig_k'].between(1e-16, 1e5)#*qs.loc[[0.02, 0.99],'sig_k'])
    sig_a_sel = df['sig_a'].between(1e-16,1e5)
    df = df.loc[sig_k_sel&sig_a_sel]
    if n==1:
        df['k'] = df['k']*-1
    # hrange =(0, srs_params['k'].quantile([0.98]).item())
    hranges = ([-0.02,0.05], [1e-8,10])
    for i,x in enumerate('ka'):

        ax[n,i].hist(df[x].values, bins=100, range=hranges[i])
        ax[n,i].axvline(df[x].median(), color='k', alpha=0.5, label=f'Median = {df[x].median():0.3g}')
        ax[n,i].legend()
    ax[n,0].set_xlabel(r"Uptake Rate  $\left(min^{-1}\right)$")
    ax[n,1].set_xlabel(r"Amplitude $\left( a.u. \right)$")

    label = ["direct", "log-linear", "uptake"][n]
    ax[n,0].set_ylabel(label)

    fig.suptitle("Estimated Single-Cell Metabolic Parameters")
    print(label)
    print(df.mean())
    print(df.median())
    print()

In [ ]:
wt_sel = wt_params['sig_k'].between(1e-16, 1e5)&wt_params['sig_a'].between(1e-16, 1e5)

In [ ]:
yup = wt_traces.loc[wt_sel.loc[wt_sel].index]

In [ ]:
dil_sel = dil_params_lin['k']<=0

In [ ]:
ydil = dil_traces.loc[dil_sel[dil_sel].index,:35]

In [ ]:
fit_up = curve_fit(exp_approach,t_wt, yup.mean()-yup.mean()[0], sigma=yup.sem(), p0=[20,0.05])

In [ ]:
pred_up = exp_approach(xplt, *fit_up[0])+yup.mean()[0]

In [ ]:
lr_dil = linregress(t_wt,np.log(ydil.mean().values))
pred_dil = np.exp(lr_dil.intercept+lr_dil.slope*xplt)

In [ ]:
Aup = fit_up[0][0]
kup = fit_up[0][1]
Adil = np.exp(lr_dil.intercept)
kdil = -lr_dil.slope

In [ ]:
plt.figure()
gr_lrs = []
for i,a in enumerate([wt_count, dil_count]):
    # count = a.stack('T').loc[a.stack('T')>0].groupby('T').count().values[:36]
    y = a.values[:36]
    lr = linregress(t_wt, np.log(y))
    gr_lrs.append(lr)
    s = ['Deuterated', 'Unlabeled'][i]
    plt.plot(t_wt, y, '.', label=f"{s} medium: Growth rate $\gamma$ = {lr.slope:0.4g}", color=colors[i])
    # plt.plot(xplt, 2**(lr.intercept+lr.slope*xplt), color=colors[i], alpha=0.6)
    plt.plot(xplt, np.exp((lr.intercept+lr.slope*xplt)), color=colors[i], alpha=0.6)
plt.semilogy()
plt.title("Population Growth")
plt.xlabel("Time (minutes)")
plt.ylabel("Number of cells")
plt.legend();

In [ ]:
plt.figure()
plt.errorbar(t_wt, yup.mean(), yerr=yup.sem(),fmt='.', capsize=3)
plt.plot(xplt, pred_up, color=colors[0], label=f"A={Aup:0.3g} k={kup:0.3g}")
plt.errorbar(t_wt, ydil.mean(),yerr=ydil.sem(),fmt='.', capsize=3)
plt.plot(xplt, pred_dil, color=colors[1], label=f"A={Adil:0.3g} k={kdil:0.3g}")
plt.legend()
plt.title("Average SRS Intensity")

In [ ]:
Nup = np.exp(gr_lrs[0].intercept)
gup = gr_lrs[0].slope
Ndil = np.exp(gr_lrs[1].intercept)
gdil = gr_lrs[1].slope

$$ \frac{dM}{dt} = N \phi_{cell}$$
$$ M = N m_0$$

$$ \frac{dN}{dt} =  \frac{\phi_{cell}}{m_0}N$$

In [ ]:
print(f"{Aup=:0.3g} {kup=:0.3g}")
print(f"{Adil=:0.3g} {kdil=:0.3g}")

In [ ]:
Aup*kup

In [ ]:
Adil*kdil

In [ ]:
print(f"{Nup=:0.3g} {gup=:0.3g}")
print(f"{Ndil=:0.3g} {gdil=:0.3g}")

In [ ]:
(kup-gup)/gup

In [ ]:
rup = kup/gup
rup

In [ ]:
1/rup

In [ ]:
rdil = kdil/gdil
rdil

In [ ]:
1/rdil

In [ ]:
rup/rdil

In [ ]:
(1-np.exp(-1))/np.exp(-1)

In [ ]:
kup/kdil

In [ ]:
gup/gdil

In [ ]:
rup2 = np.array([1.0386548516730536, 1.5492932717898884, 1.5506088069480395, 1.626218714457804, 1.2748807840158445])
rdil2 = np.array([0.8166119942118373, 0.7727661021075077, 0.7393169821046168, 0.8182849806331856, 0.7584506687997252])

In [ ]:
np.mean(rup2)/np.mean(rdil2)

In [ ]:
np.mean(rup2)/np.mean(rdil2)

In [ ]:
np.mean(rup2)

In [ ]:
np.mean(rdil2)

In [ ]:
np.sqrt(np.var(rup2)/(np.mean(rup2)**2) + np.var(rdil2)/(np.mean(rdil2)**2))

In [ ]:
first_areas = wt_area.stack('T').dropna().reset_index('T').groupby(['S','CellID']).first()

In [ ]:
growth_areas = wt_area.loc[first_areas[0]<vox_area*100]

In [ ]:
aligned_areas = np.full_like(growth_areas.values, np.nan)
for i,(_,a) in enumerate(growth_areas.iterrows()):
    x = a.dropna().values
    aligned_areas[i,:a.shape[0]] = a

In [ ]:
n = aligned_areas.shape[0]-np.isnan(aligned_areas).sum(0)

In [ ]:
v0 = wt_area[0].dropna().mean()
vsem  = wt_area[0].dropna().std()

In [ ]:
plt.figure()
plt.plot(aligned_areas.T, 'k', alpha=0.1);

In [ ]:
plt.figure()
for x,a in growth_areas.iterrows():
    plt.plot(a.dropna().values, 'k', alpha=0.05)

In [ ]:
tbinned_areas = growth_areas.groupby(first_areas['T']).mean()
tbinned_var = growth_areas.groupby(first_areas['T']).var()

In [ ]:
tbinned_srs = wt_traces.groupby(first_areas['T']).mean()

In [ ]:
vcolors = plt.cm.viridis(np.linspace(0.05, 0.95, tbinned_areas.shape[0]))

In [ ]:
(first_areas['T'].max()-first_areas['T'].min())/0.9 + 0.05

In [ ]:
aligned_areas = np.full_like(tbinned_areas.values, np.nan)
aligned_vars = np.full_like(aligned_areas, np.nan)
for i in range(tbinned_areas.shape[0]):
    x = tbinned_areas.iloc[i].dropna().values
    aligned_areas[i,:x.shape[0]] = x
    x = tbinned_var.iloc[i].dropna().values
    aligned_vars[i,:x.shape[0]] = x
aligned_areas = aligned_areas[:,:-1]
aligned_vars = aligned_vars[:,:-1]

In [ ]:
aligned_srs = np.full_like(tbinned_srs.values, np.nan)
for i in range(tbinned_srs.shape[0]):
    x = tbinned_srs.iloc[i].dropna().values
    aligned_srs[i,:x.shape[0]] = x-x[0]
aligned_srs = aligned_srs[:,:-1]

In [ ]:
plt.figure()
for i,(x,a) in enumerate(tbinned_areas.iterrows()):
    y = a.dropna().values
    plt.plot(t_wt[:y.shape[0]],y, color=vcolors[i], alpha=0.5)
# plt.legend(title="Time of birth (minutes)")
plt.plot(t_wt[:-1], np.nanmean(aligned_areas, axis=0),'k', label='Average', linewidth=2)
cb = plt.colorbar(plt.cm.ScalarMappable( cmap=plt.cm.viridis), label="Time of birth (minutes)")
cb.ax.set_yticklabels([f"{t_wt[21]*x:0.1f}" for x in np.linspace(0,1,6)])
plt.ylabel("Area $\\left(\mu m^2 \\right)$")
plt.xlabel("Time since birth (minutes)")
plt.title("Cellular Growth")
plt.axhline(v0, color='k',linestyle='--', alpha=0.5, label="Average size of initial cells")
# plt.errorbar(1./0.004, v0,yerr=vsem, fmt='ro', capsize=3)
plt.legend()
# plt.semilogy();
if savefigs: plt.savefig(figpath+"area_growth.png")

In [ ]:
plt.figure()
for i,(x,a) in enumerate(tbinned_srs.iterrows()):
    y = a.dropna().values
    plt.plot(t_wt[-y.shape[0]:],y, color=vcolors[i], alpha=0.75)
# plt.legend(title="Time of birth (minutes)")
plt.plot(t_wt,tbinned_srs.mean(),'k', label='Average', linewidth=2)
cb = plt.colorbar(plt.cm.ScalarMappable( cmap=plt.cm.viridis), label="Time of birth (minutes)")
cb.ax.set_yticklabels([f"{t_wt[21]*x:0.1f}" for x in np.linspace(0,1,6)])
plt.ylabel("SRS Intensity (a.u.)")
plt.xlabel("Time (minutes)")
plt.title("Net-glucose uptake after cell birth")
# plt.axhline(v0, color='k',linestyle='--', alpha=0.5, label="Average size of initial cells")
# plt.errorbar(1./0.004, v0,yerr=vsem, fmt='ro', capsize=3)
plt.legend()
# plt.semilogy();
if savefigs: plt.savefig(figpath+"srs_by_age.png")

In [ ]:
plt.figure()
for i,(x,a) in enumerate(tbinned_srs.iterrows()):
    y = a.dropna().values
    plt.plot(t_wt[:y.shape[0]],y-y[0], color=vcolors[i], alpha=0.75)
# plt.legend(title="Time of birth (minutes)")
# plt.plot(t_wt[:-1], np.nanmean(aligned_srs,0),'k', label='Average', linewidth=2)
cb = plt.colorbar(plt.cm.ScalarMappable( cmap=plt.cm.viridis), label="Time of birth (minutes)")
cb.ax.set_yticklabels([f"{t_wt[21]*x:0.1f}" for x in np.linspace(0,1,6)])
plt.title("Net Glucose Uptake After Birth")
plt.xlabel("Time since birth (minutes)")
plt.ylabel("SRS (a.u.)")
# plt.axhline(v0, color='k',linestyle='--', alpha=0.5, label="Average size of initial cells")
# plt.errorbar(1./0.004, v0,yerr=vsem, fmt='ro', capsize=3)
# plt.legend()
# plt.semilogy();
if savefigs: plt.savefig(figpath+"aligned_uptake.png")